# Deep Reinforcement Learning using AlphaZero methodology

Please see https://applied-data.science/blog/how-to-build-your-own-alphazero-ai-using-python-and-keras/ for further notes on the codebase

这个环境在使用tensorflow的docker时，可能有点问题，需要先安装一点包。首先，需要这样运行docker:
```
docker run -ti --name beta-zero -e GRANT_SUDO=yes --user root -p 8888:8888 -v "$PWD":/home/jovyan/work jupyter/tensorflow-notebook
```
然后，进入容器中，安装如下包：
```
apt-get update
apt-get install python-pydot python-pydot-ng graphviz
```

然后，再执行如下代码。之后，就可以开始训练了。注意，如果有新的训练版本，需要将它们copy到配置指定的archive目录，以便下次学习时继续。
例如，执行这样的命令：
```
-SERVER:~/todo/alpha/DeepReinforcementLearning$ cp - run/models archive/connect4/run0001/.
-SERVER:~/todo/alpha/DeepReinforcementLearning$ cp - run/memory archive/connect4/run0001/.
```
然后，修改initialise.py：
```
INITIAL_MODEL_VERSION = 21
INITIAL_MEMORY_VERSION = 75
```
将model及memory设置为当前最高的版本号，就可以继续学习了。

In [1]:
!pip install --upgrade pip
!pip install pydot graphviz

Requirement already up-to-date: pip in /opt/conda/lib/python3.6/site-packages (10.0.1)
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
ipywidgets 7.0.5 has requirement widgetsnbextension~=3.0.0, but you'll have widgetsnbextension 3.1.3 which is incompatible.
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
ipywidgets 7.0.5 has requirement widgetsnbextension~=3.0.0, but you'll have widgetsnbextension 3.1.3 which is incompatible.


## 1. First load the core libraries

In [2]:
# -*- coding: utf-8 -*-
# %matplotlib inline

import numpy as np
np.set_printoptions(suppress=True)

from shutil import copyfile
import random


from keras.utils import plot_model

from game import Game, GameState
from agent import Agent
from memory import Memory
from model import Residual_CNN
from funcs import playMatches, playMatchesBetweenVersions

import loggers as lg

from settings import run_folder, run_archive_folder
import initialise
import pickle
import importlib
print ('I am ready.')

Using TensorFlow backend.


I am ready.


## 2. Now run this block to start the learning process

This block loops for ever, continually learning from new game data.

The current best model and memories are saved in the run folder so you can kill the process and restart from the last checkpoint.

In [ ]:
lg.logger_main.info('=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*')
lg.logger_main.info('=*=*=*=*=*=.      NEW LOG      =*=*=*=*=*')
lg.logger_main.info('=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*')

env = Game()

# If loading an existing neural network, copy the config file to root
if initialise.INITIAL_RUN_NUMBER != None:
    copyfile(run_archive_folder + env.name + '/run' + str(initialise.INITIAL_RUN_NUMBER).zfill(4) + '/config.py', './config.py')

import config

######## LOAD MEMORIES IF NECESSARY ########

if initialise.INITIAL_MEMORY_VERSION == None:
    memory = Memory(config.MEMORY_SIZE)
else:
    print('LOADING MEMORY VERSION ' + str(initialise.INITIAL_MEMORY_VERSION) + '...')
    memory = pickle.load( open( run_archive_folder + env.name + '/run' + str(initialise.INITIAL_RUN_NUMBER).zfill(4) + "/memory/memory" + str(initialise.INITIAL_MEMORY_VERSION).zfill(4) + ".p",   "rb" ) )

######## LOAD MODEL IF NECESSARY ########

# create an untrained neural network objects from the config file
current_NN = Residual_CNN(config.REG_CONST, config.LEARNING_RATE, (2,) + env.grid_shape,   env.action_size, config.HIDDEN_CNN_LAYERS)
best_NN = Residual_CNN(config.REG_CONST, config.LEARNING_RATE, (2,) +  env.grid_shape,   env.action_size, config.HIDDEN_CNN_LAYERS)

#If loading an existing neural netwrok, set the weights from that model
if initialise.INITIAL_MODEL_VERSION != None:
    best_player_version  = initialise.INITIAL_MODEL_VERSION
    print('LOADING MODEL VERSION ' + str(initialise.INITIAL_MODEL_VERSION) + '...')
    m_tmp = best_NN.read(env.name, initialise.INITIAL_RUN_NUMBER, best_player_version)
    current_NN.model.set_weights(m_tmp.get_weights())
    best_NN.model.set_weights(m_tmp.get_weights())
#otherwise just ensure the weights on the two players are the same
else:
    best_player_version = 0
    best_NN.model.set_weights(current_NN.model.get_weights())

#copy the config file to the run folder
copyfile('./config.py', run_folder + 'config.py')
plot_model(current_NN.model, to_file=run_folder + 'models/model.png', show_shapes = True)

print('\n')

######## CREATE THE PLAYERS ########

current_player = Agent('current_player', env.state_size, env.action_size, config.MCTS_SIMS, config.CPUCT, current_NN)
best_player = Agent('best_player', env.state_size, env.action_size, config.MCTS_SIMS, config.CPUCT, best_NN)
#user_player = User('player1', env.state_size, env.action_size)
iteration = 0

while 1:

    iteration += 1
    importlib.reload(lg)
    importlib.reload(config)
    
    print('ITERATION NUMBER ' + str(iteration))
    
    lg.logger_main.info('BEST PLAYER VERSION: %d', best_player_version)
    print('BEST PLAYER VERSION ' + str(best_player_version))

    ######## SELF PLAY ########
    print('SELF PLAYING ' + str(config.EPISODES) + ' EPISODES...')
    _, memory, _, _ = playMatches(best_player, best_player, config.EPISODES, lg.logger_main, turns_until_tau0 = config.TURNS_UNTIL_TAU0, memory = memory)
    print('\n')
    
    memory.clear_stmemory()
    
    if len(memory.ltmemory) >= config.MEMORY_SIZE:

        ######## RETRAINING ########
        print('RETRAINING...')
        current_player.replay(memory.ltmemory)
        print('')

        if iteration % 5 == 0:
            pickle.dump( memory, open( run_folder + "memory/memory" + str(iteration).zfill(4) + ".p", "wb" ) )

        lg.logger_memory.info('====================')
        lg.logger_memory.info('NEW MEMORIES')
        lg.logger_memory.info('====================')
        
        memory_samp = random.sample(memory.ltmemory, min(1000, len(memory.ltmemory)))
        
        for s in memory_samp:
            current_value, current_probs, _ = current_player.get_preds(s['state'])
            best_value, best_probs, _ = best_player.get_preds(s['state'])

            lg.logger_memory.info('MCTS VALUE FOR %s: %f', s['playerTurn'], s['value'])
            lg.logger_memory.info('CUR PRED VALUE FOR %s: %f', s['playerTurn'], current_value)
            lg.logger_memory.info('BES PRED VALUE FOR %s: %f', s['playerTurn'], best_value)
            lg.logger_memory.info('THE MCTS ACTION VALUES: %s', ['%.2f' % elem for elem in s['AV']]  )
            lg.logger_memory.info('CUR PRED ACTION VALUES: %s', ['%.2f' % elem for elem in  current_probs])
            lg.logger_memory.info('BES PRED ACTION VALUES: %s', ['%.2f' % elem for elem in  best_probs])
            lg.logger_memory.info('ID: %s', s['state'].id)
            lg.logger_memory.info('INPUT TO MODEL: %s', current_player.model.convertToModelInput(s['state']))

            s['state'].render(lg.logger_memory)
            
        ######## TOURNAMENT ########
        print('TOURNAMENT...')
        scores, _, points, sp_scores = playMatches(best_player, current_player, config.EVAL_EPISODES, lg.logger_tourney, turns_until_tau0 = 0, memory = None)
        print('\nSCORES')
        print(scores)
        print('\nSTARTING PLAYER / NON-STARTING PLAYER SCORES')
        print(sp_scores)
        #print(points)

        print('\n\n')

        if scores['current_player'] > scores['best_player'] * config.SCORING_THRESHOLD:
            best_player_version = best_player_version + 1
            best_NN.model.set_weights(current_NN.model.get_weights())
            best_NN.write(env.name, best_player_version)

    else:
        print('MEMORY SIZE: ' + str(len(memory.ltmemory)))

LOADING MEMORY VERSION 75...
LOADING MODEL VERSION 21...


ITERATION NUMBER 1
BEST PLAYER VERSION 21
SELF PLAYING 25 EPISODES...
1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 


## The following panels are not involved in the learning process

### Play matches between versions (use -1 for human player)
下面的代码，是让第1版本和第21版本下5盘，在训练时好像不能运行。需要执行第1段代码。
一个结果示例如下：
```
({'drawn': 0, 'player1': 0, 'player2': 5},
 None,
 {'player1': [-1, -1, -1, -1, -1], 'player2': [1, 1, 1, 1, 1]},
 {'drawn': 0, 'nsp': 3, 'sp': 2})
 ```
 
 从上面看出，和了0盘，player1胜了0盘，player2胜了5盘。
 nsp和sp是什么意思？

In [2]:
from game import Game
from funcs import playMatchesBetweenVersions
import loggers as lg

env = Game()
playMatchesBetweenVersions(env, 1, 1, 21, 5, lg.logger_tourney, 0)

1 
2 
3 
4 
5 


({'drawn': 0, 'player1': 0, 'player2': 5},
 None,
 {'player1': [-1, -1, -1, -1, -1], 'player2': [1, 1, 1, 1, 1]},
 {'drawn': 0, 'nsp': 1, 'sp': 4})

### Pass a particular game state through the neural network (setup below for Connect4)

初始化棋盘时，7个位置，预测的概率一样：
```
(array([ 0.], dtype=float32), array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.14285715,  0.14285715,  0.14285715,  0.14285715,  0.14285715,
        0.14285715,  0.14285715], dtype=float32), [35, 36, 37, 38, 39, 40, 41])
```

In [7]:
gs = GameState(np.array([
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0
]), 1)

env = Game()
copyfile('./config.py', run_folder + 'config.py')
copyfile('./config.py', run_archive_folder + 'config.py')

# If loading an existing neural network, copy the config file to root
print ('initialise.INITIAL_RUN_NUMBER: ', initialise.INITIAL_RUN_NUMBER)
if initialise.INITIAL_RUN_NUMBER != None:
    print (run_archive_folder + env.name + '/run' + str(initialise.INITIAL_RUN_NUMBER).zfill(4) + '/config.py')
    copyfile('./config.py', run_archive_folder + env.name + '/run' + str(initialise.INITIAL_RUN_NUMBER).zfill(4) + '/config.py' )

import config

current_NN = Residual_CNN(config.REG_CONST, config.LEARNING_RATE, (2,) + env.grid_shape,   env.action_size, config.HIDDEN_CNN_LAYERS)

current_player = Agent('current_player', env.state_size, env.action_size, config.MCTS_SIMS, config.CPUCT, current_NN)


preds = current_player.get_preds(gs)

print(preds)

initialise.INITIAL_RUN_NUMBER:  None
(array([ 0.], dtype=float32), array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.14285715,  0.14285715,  0.14285715,  0.14285715,  0.14285715,
        0.14285715,  0.14285715], dtype=float32), [35, 36, 37, 38, 39, 40, 41])


# 下面使用最新版本来预测一个特定棋盘

这个棋盘，最底下7个位置中，左右双方各占3子。即谁走中间，谁就赢了。看看NN如何预测。
当参数未配置时，使用默认未训练的版本进行评估：(initialise.py:)
```
INITIAL_RUN_NUMBER = None
INITIAL_MODEL_VERSION = None
INITIAL_MEMORY_VERSION = None
```
结果：
```
You need set best NN!!!
initialise.INITIAL_RUN_NUMBER:  None
(array([-0.0025973], dtype=float32), array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.15512991,  0.15597786,
        0.14247645,  0.        ,  0.1319422 ,  0.12160993,  0.13848288,
        0.        ,  0.        ,  0.        ,  0.15438075,  0.        ,
        0.        ,  0.        ], dtype=float32), [28, 29, 30, 32, 33, 34, 38])
```
设置为：
```
INITIAL_RUN_NUMBER = 1
INITIAL_MODEL_VERSION = 21
INITIAL_MEMORY_VERSION = 75
```

重启kernel? 生效后，结果如下：
```
LOADING MODEL VERSION 21...
initialise.INITIAL_RUN_NUMBER:  1
./archive/connect4/run0001/config.py
(array([ 0.0151987], dtype=float32), array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.06248605,  0.11054417,
        0.24719164,  0.        ,  0.17689258,  0.09741932,  0.07058962,
        0.        ,  0.        ,  0.        ,  0.2348766 ,  0.        ,
        0.        ,  0.        ], dtype=float32), [28, 29, 30, 32, 33, 34, 38])
```

当然，这个只是认为必胜的那个位置感觉好，但还不是最高。毕竟这个只是棋感，不是计算。
只是感觉它好，是否胜，还需要计算。



In [2]:
import config
env = Game()

best_NN = Residual_CNN(config.REG_CONST, config.LEARNING_RATE, (2,) +  env.grid_shape,   env.action_size, config.HIDDEN_CNN_LAYERS)

if initialise.INITIAL_MODEL_VERSION != None:
    best_player_version  = initialise.INITIAL_MODEL_VERSION
    print('LOADING MODEL VERSION ' + str(initialise.INITIAL_MODEL_VERSION) + '...')
    m_tmp = best_NN.read(env.name, initialise.INITIAL_RUN_NUMBER, best_player_version)
    # current_NN.model.set_weights(m_tmp.get_weights())
    best_NN.model.set_weights(m_tmp.get_weights())
    #otherwise just ensure the weights on the two players are the same
else:
    print ('You need set best NN!!!')
    
gs = GameState(np.array([
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    1,1,1,0,-1,-1,-1
]), 1)


env = Game()
copyfile('./config.py', run_folder + 'config.py')
copyfile('./config.py', run_archive_folder + 'config.py')

# If loading an existing neural network, copy the config file to root
print ('initialise.INITIAL_RUN_NUMBER: ', initialise.INITIAL_RUN_NUMBER)
if initialise.INITIAL_RUN_NUMBER != None:
    print (run_archive_folder + env.name + '/run' + str(initialise.INITIAL_RUN_NUMBER).zfill(4) + '/config.py')
    copyfile('./config.py', run_archive_folder + env.name + '/run' + str(initialise.INITIAL_RUN_NUMBER).zfill(4) + '/config.py' )

import config

# current_NN = Residual_CNN(config.REG_CONST, config.LEARNING_RATE, (2,) + env.grid_shape,   env.action_size, config.HIDDEN_CNN_LAYERS)

# current_player = Agent('current_player', env.state_size, env.action_size, config.MCTS_SIMS, config.CPUCT, best_NN)
best_player = Agent('best_player', env.state_size, env.action_size, config.MCTS_SIMS, config.CPUCT, best_NN)

preds = best_player.get_preds(gs)

print(preds)

LOADING MODEL VERSION 21...
initialise.INITIAL_RUN_NUMBER:  1
./archive/connect4/run0001/config.py
(array([ 0.0151987], dtype=float32), array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.06248605,  0.11054417,
        0.24719164,  0.        ,  0.17689258,  0.09741932,  0.07058962,
        0.        ,  0.        ,  0.        ,  0.2348766 ,  0.        ,
        0.        ,  0.        ], dtype=float32), [28, 29, 30, 32, 33, 34, 38])


### See the layers of the current neural network

In [ ]:
current_player.model.viewLayers()

### Output a diagram of the neural network architecture

In [ ]:
from keras.utils import plot_model
plot_model(current_NN.model, to_file=run_folder + 'models/model.png', show_shapes = True)